In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
#Загрузка данных
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

Мы уже просмотрели данные, и понимаем что пропущенны значения возраста, класса. А также удалим значения, которые никак не повлияют на выбор или пропущены, так много раз, что мы не сможем их восстановить.

In [4]:
# Заполнение пропущенных значений возраста средним значением
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)

# Заполнение пропущенных значений тарифа средним значением
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)

# Разделяем пол и классы на 0/1
train_data = pd.get_dummies(train_data, columns=['Sex', 'Embarked'])
test_data = pd.get_dummies(test_data, columns=['Sex', 'Embarked'])

In [5]:
# Смысла нет в обучении от имени
train_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [6]:
train_data.isnull().sum(), test_data.isnull().sum()

(PassengerId    0
 Survived       0
 Pclass         0
 Age            0
 SibSp          0
 Parch          0
 Fare           0
 Sex_female     0
 Sex_male       0
 Embarked_C     0
 Embarked_Q     0
 Embarked_S     0
 dtype: int64,
 PassengerId    0
 Pclass         0
 Age            0
 SibSp          0
 Parch          0
 Fare           0
 Sex_female     0
 Sex_male       0
 Embarked_C     0
 Embarked_Q     0
 Embarked_S     0
 dtype: int64)

In [7]:
X_train = train_data.drop(['Survived', 'PassengerId'], axis=1)  # Удаляем столбцы, которые не являются признаками
y_train = train_data['Survived']  # Целевая переменная

In [8]:
#Нормализация данных
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test = test_data.drop(['PassengerId'], axis=1)

X_test_scaled = scaler.transform(X_test)

Обучение модели, вариант 1

In [9]:
knn_model = KNeighborsClassifier(n_neighbors=2)

# Обучение модели
knn_model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=2)

In [10]:
result = knn_model.predict(X_test_scaled)

sub = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": result
})

sub.to_csv("titatic_v1.csv", index=False)

Результат был 0.72009, что хотелось увеличить
# Версия V2



Используем GridSearchCV для поиска оптимальных параметров.

In [21]:
# Определение параметров и их значений для перебора
grid = {'n_neighbors': range(1, 100),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_scaled, y_train)

grid_search.best_params_

{'metric': 'manhattan', 'n_neighbors': 14, 'weights': 'uniform'}

In [22]:
# Обучаем модель с использованием новых параметров
knn_model = KNeighborsClassifier(n_neighbors=14, weights='uniform', metric='manhattan')
knn_model.fit(X_train_scaled, y_train)

result = knn_model.predict(X_test_scaled)

In [23]:
result = knn_model.predict(X_test_scaled)

sub = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": result
})

sub.to_csv("titatic_v2.csv", index=False)